# idealized Geometry
using the IdealGeometry class

In [ ]:
import os
import sys
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

sys.path.append('..')
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
from forcing import Forcing
from ideal_geometry import IdealGeometry, FavierTest

## ISOMIP geometry

In [ ]:
ds = IdealGeometry('test1').create()

In [ ]:
ds.draft.plot()

In [ ]:
ds1 = xr.open_dataset('../../data/isomip/Ocean1_input_geom_v1.01.nc')
ds2 = xr.open_dataset('../../data/isomip/Ocean2_input_geom_v1.01.nc')
# ds3 = xr.open_dataset('../../data/isomip/Ocean3_input_geom_v1.01.nc')
# ds4 = xr.open_dataset('../../data/isomip/Ocean4_input_geom_v1.01.nc')
plt.figure(figsize=(12,5))
plt.axhline(0,c='grey',lw=.5)
ds1.bedrockTopography.sel(y=40000,method='nearest').plot(c='k')
for i, ds in enumerate([ds1,ds2]):  #,ds3.isel(t=0),ds3.isel(t=-1),ds4.isel(t=0),ds4.isel(t=-1)]):
    name = ['Ocean1','Ocean2','Ocean3 start','Ocean3 end','Ocean4 start','Ocean4 end'][i]
    c = f'C{i}'
    ds_ = ds.sel(y=40000,method='nearest')
    (ds_.upperSurface+i*20).plot(c=c, label=name)
    (ds_.lowerSurface+i*20).plot(c=c)
plt.title('ice shelf centre lines')
plt.text(700000,-150,'offsets for visual clarity')
plt.ylabel('z [m]')
plt.legend()

In [ ]:
dg1 = IdealGeometry('Ocean1').create()
dg2 = IdealGeometry('Ocean2').create()

quantities = ['draft','mask','dgrl','disf','box','alpha']
f, ax = plt.subplots(2, len(quantities), figsize=(15,5), sharex=True, sharey=True)
for q, Q in enumerate(quantities):
    dg1[Q].plot(ax=ax[0,q], add_colorbar=False)
    dg2[Q].plot(ax=ax[1,q], add_colorbar=False)
    ax[0,q].set_title(Q)

In [ ]:
dg1.alpha.plot(vmax=.00000003, figsize=(15,10))

In [ ]:
dg2.grl_adv.plot(figsize=(15,10))

## idealized test geometries

In [ ]:
f, ax = plt.subplots(6,3, figsize=(8,8), sharex=True, sharey=True, constrained_layout=True)
for i in range(3):
    if i==2:
        cbargs = {}
    else:
        cbargs = dict(add_colorbar=False)
    ax[-1,i].set_xlabel('x [km]')
    tg = IdealGeometry(f'test{i+1}').create()
    tg = tg.assign_coords({'x':tg.x/1e3, 'y':tg.y/1e3})
    tg.draft  .plot(ax=ax[0,i], vmin=-1000, vmax=-500 , **cbargs)
    tg.mask   .plot(ax=ax[1,i]                        , **cbargs)
    tg.alpha  .plot(ax=ax[2,i], vmin=    0, vmax=0.006, **cbargs)
    tg.dgrl   .plot(ax=ax[3,i]                        , **cbargs)
    tg.grl_adv.plot(ax=ax[4,i], vmin=-1000, vmax=-500 , **cbargs)
    tg.box    .plot(ax=ax[5,i]                        , **cbargs)
for j in range(5):
    ax[j,0].set_ylabel('y [km]')
    

In [ ]:
tg

In [ ]:
ds = Forcing(tg).tanh(ztcl=-700,Tdeep=0)

In [ ]:
f, ax = plt.subplots(1,2,figsize=(10,3),constrained_layout=True)
ax[0].plot(ds.Tz, ds.z)
ax[0].set_ylim((-1500,0))
ds.Ta.plot(ax=ax[1])

In [ ]:
ds_ = Forcing(tg).constant()

In [ ]:
f, ax = plt.subplots(1,2,figsize=(10,3),constrained_layout=True)
ax[0].plot(ds_.Tz, ds_.z)
ax[0].set_ylim((-1500,0))
ds_.Ta.plot(ax=ax[1])

## geometries and forcing scenarios of Favier et al. (2019)

In [ ]:
iceshelves = ['fris', 'totten', 'thwaites', 'test', 'test2']

f, ax = plt.subplots(3, len(iceshelves), figsize=(10,6), sharex=True, sharey=True)

for i, iceshelf in enumerate(iceshelves):
    if i==len(iceshelves)-1:
        cbarargs = {}
    else:
        cbarargs = dict(add_colorbar=False)
    ds = FavierTest(iceshelf, 'warm0')
    ds.draft.plot(ax=ax[0,i], vmin=-1000, vmax=0   , **cbarargs)
    ds.Ta   .plot(ax=ax[1,i], vmin= -1.2, vmax=1.2 , **cbarargs)
    ds.Sa   .plot(ax=ax[2,i], vmin= 34  , vmax=34.8, **cbarargs)
    ax[0,i].set_title(iceshelf)
    